## Prepare data

In [1]:
from giza_helpers.prepare_data import *
uni, eth = download_data()
X_train, X_test, Y_train, Y_test = process_data(uni, eth)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


1        4.043332
2      -52.461419
3      -17.076879
4       -6.215413
5       61.033549
          ...    
1616    -0.105024
1617    -1.990389
1618     0.386418
1619    -0.500657
1620     0.337570
Name: price, Length: 1620, dtype: float64


/Users/selim/Desktop/firstbatch/giza/giza-notebooks/giza_helpers/prepare_data.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth.rename(columns={"Open": "ETH"}, inplace=True)


In [2]:
X_train

,realized_vol,returns_squared
0,41.234377,3725.094061
1,41.524274,94.512324
2,29.948475,168.159335
3,25.666232,58.146380
4,22.469835,201.413137
...,...,...
1354,1.629951,12.127320
1355,1.616969,0.205096
1356,1.686993,0.148979
1357,1.552290,0.003271


## Build and train the model

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error as mse
import time


def train_model(
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    Y_train: pd.DataFrame,
    Y_test: pd.DataFrame,
):
    model = nn.Sequential(
        nn.Linear(X_train.shape[1], 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 1),
    )

    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.RMSprop(model.parameters())

    # Convert data to PyTorch tensors
    X_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_tensor = torch.tensor(Y_train.values.reshape(-1, 1), dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

    # Training loop
    epochs_trial = np.arange(100, 400, 4)
    batch_trial = np.arange(100, 400, 4)
    DL_pred = []
    DL_RMSE = []

    for i, j, k in zip(range(4), epochs_trial, batch_trial):
        for epoch in range(j):
            optimizer.zero_grad()
            outputs = model(X_tensor)
            loss = criterion(outputs, y_tensor)
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            DL_predict = model(X_test_tensor).numpy()
            DL_RMSE.append(
                np.sqrt(mse(Y_test.values / 100, DL_predict.flatten() / 100))
            )
            DL_pred.append(DL_predict)
            print("DL_RMSE_{}:{:.6f}".format(i + 1, DL_RMSE[i]))

    return model


def serialize_to_onnx(
    model: nn.Module, X_train: pd.DataFrame, save_path="uniswap_lp_nn_model"
):
    model.eval()

    sample_input = torch.randn(
        1, X_train.shape[1]
    )

    onnx_file_path = save_path + ".onnx"

    torch.onnx.export(
        model,
        sample_input,
        onnx_file_path,
        export_params=True,
        opset_version=10,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size"},
            "output": {0: "batch_size"},
        },
    )
    print(f"Saved serialized ONNX model to {onnx_file_path}.")



In [8]:
model = train_model(X_train, X_test, Y_train, Y_test)
serialize_to_onnx(model, X_train)

DL_RMSE_1:0.566944
DL_RMSE_2:0.568271
DL_RMSE_3:0.680286
DL_RMSE_4:0.646960
Saved serialized ONNX model to uniswap_lp_nn_model.onnx.


## Transpile and create the endpoint

In [9]:
# transpile the model into Orion Cairo

start_time = time.time()
! giza transpile uniswap_lp_nn_model.onnx --output-path uniswap_lp_nn
end_time = time.time()

print(f"Time taken for the operation: {end_time - start_time} seconds")

[giza][2024-07-01 10:49:34.911] No model id provided, checking if model exists ✅
[giza][2024-07-01 10:49:34.912] Model name is: uniswap_lp_nn_model
[giza][2024-07-01 10:49:35.043] Model already exists, using existing model ✅ 
[giza][2024-07-01 10:49:35.044] Model found with id -> 857! ✅
[giza][2024-07-01 10:49:35.490] Version Created with id -> 10! ✅
[giza][2024-07-01 10:49:35.491] Sending model for transpilation ✅ 
[giza][2024-07-01 10:50:26.506] Transpilation is fully compatible. Version compiled and Sierra is saved at Giza ✅
⠼ Transpiling Model...
[giza][2024-07-01 10:50:27.284] Downloading model ✅
[giza][2024-07-01 10:50:27.295] model saved at: uniswap_lp_nn
                   Model                   
┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ id  ┃ name                ┃ description ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 857 │ uniswap_lp_nn_model │             │
└─────┴─────────────────────┴─────────────┘
                                    Version                      

In [11]:
# create giza endpoint
! giza endpoints deploy --model-id 857 --version-id 3

▰▰▰▰▱▱▱ Creating endpoint!t!
[giza][2024-06-28 18:04:28.465] Endpoint is successful ✅
[giza][2024-06-28 18:04:28.467] Endpoint created with id -> 411 ✅
[giza][2024-06-28 18:04:28.467] Endpoint created with endpoint URL: https://endpoint-selimsheker-857-3-3f40ae83-7i3yxzspbq-ew.a.run.app 🎉


In [45]:
MODEL_ID = 857
VERSION_ID = 3
ENDPOINT_ID = 411

# Run verifiable inference using giza sdk

In [8]:
import xgboost as xgb
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

from giza.agents.model import GizaModel

In [43]:
def prediction(input, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, proof_id) = model.predict(
        input_feed={"input": input}, verifiable=True, model_category="ONNX_ORION", dry_run=False
    )

    return result, proof_id


def execution():
    input = X_test.iloc[0].to_numpy().reshape(1,2)

    (result, proof_id) = prediction(input, MODEL_ID, VERSION_ID)

    print(f"Predicted value for input {input.flatten()} is {result}")

    return result, proof_id

In [44]:
result, proof_id = execution()
print(f"Proof ID: {proof_id}")

🚀 Starting deserialization process...
✅ Deserialization completed! 🎉
Predicted value for input [3.22083252e-01 1.66206885e-04] is [[11.82849121]]
Proof ID: fd230f906fce43ab9ffcc8096e944da7


In [51]:
PROOF_ID = "fd230f906fce43ab9ffcc8096e944da7"

# Verify the proof

In [52]:
# get proof
! giza endpoints get-proof --endpoint-id 411 --proof-id "{PROOF_ID}"

[giza][2024-06-30 18:49:20.368] Getting proof from endpoint 411 ✅ 
┏━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ id   ┃ job_id ┃ proving_t… ┃ cairo_exe… ┃ metrics   ┃ created_d… ┃ request_… ┃
┡━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ 1273 │ 1465   │            │            │ {'provin… │ 2024-06-30 │           │
│      │        │            │            │ 34.41734… │ 15:47:34.… │           │
└──────┴────────┴────────────┴────────────┴───────────┴────────────┴───────────┘


In [53]:
# download proof
! giza endpoints download-proof --endpoint-id {ENDPOINT_ID} --proof-id "{PROOF_ID}" --output-path uniswap_lp_nn_proof.proof

[giza][2024-06-30 18:49:34.940] Getting proof from endpoint 411 ✅ 
[giza][2024-06-30 18:49:35.984] Proof downloaded to uniswap_lp_nn_proof.proof ✅ 


In [54]:
# verify
!giza verify --proof-id 1273

[giza][2024-06-30 18:49:44.509] Verifying proof...
[giza][2024-06-30 18:49:47.002] Verification result: True
[giza][2024-06-30 18:49:47.002] Verification time: 0.971497554


## Giza agents

In [ ]:
# ! giza agents create --endpoint-id 395 --name uniswap_lp_agent --description uniswap_lp_agent

In [31]:
import argparse
import logging
import os
import pprint
from logging import getLogger

import numpy as np
from dotenv import find_dotenv, load_dotenv
from giza.agents import AgentResult, GizaAgent

from giza_helpers.addresses import ADDRESSES
from giza_helpers.lp_tools import get_tick_range
from giza_helpers.uni_helpers import (approve_token, check_allowance, close_position,
                         get_all_user_positions, get_mint_params)


load_dotenv(find_dotenv())

os.environ["GIZA-AGENT-TEST-1_PASSPHRASE"] = os.environ.get("GIZA-AGENT-TEST-1_PASSPHRASE")
sepolia_rpc_url = os.environ.get("SEPOLIA_RPC_URL")

logging.basicConfig(level=logging.INFO)


In [22]:
def process_data(realized_vol: float, dec_price_change: float):
    pct_change_sq = (100 * dec_price_change) ** 2
    X = np.array([[realized_vol, pct_change_sq]])
    return X


def get_data():
    # TODO: implement fetching onchain or from some other source
    # hardcoding the values for now
    realized_vol = 4.20
    dec_price_change = 0.1
    return realized_vol, dec_price_change


def create_agent(
    model_id: int, version_id: int, chain: str, contracts: dict, account: str
):
    """
    Create a Giza agent for the volatility prediction model
    """
    agent = GizaAgent(
        contracts=contracts,
        id=model_id,
        version_id=version_id,
        chain=chain,
        account=account,
    )
    return agent


def predict(agent: GizaAgent, X: np.ndarray):
    """
    Predict the next day volatility.

    Args:
        X (np.ndarray): Input to the model.

    Returns:
        int: Predicted value.
    """
    prediction = agent.predict(input_feed={"val": X}, verifiable=False, job_size="XL")
    return prediction


def get_pred_val(prediction: AgentResult):
    """
    Get the value from the prediction.

    Args:
        prediction (dict): Prediction from the model.

    Returns:
        int: Predicted value.
    """
    # This will block the executon until the prediction has generated the proof
    # and the proof has been verified
    return prediction.value[0][0]


def rebalance_lp(
    tokenA_amount: int,
    tokenB_amount: int,
    pred_model_id: int,
    pred_version_id: int,
    account="dev",
    chain=f"ethereum:sepolia:{sepolia_rpc_url}",
    nft_id=None,
):
    logger = getLogger("agent_logger")
    nft_manager_address = ADDRESSES["NonfungiblePositionManager"][11155111]
    tokenA_address = ADDRESSES["UNI"][11155111]
    tokenB_address = ADDRESSES["WETH"][11155111]
    pool_address = "0x287B0e934ed0439E2a7b1d5F0FC25eA2c24b64f7"
    user_address = "0xCBB090699E0664f0F6A4EFbC616f402233718152"
    pool_fee = 3000
    logger.info("Fetching input data")
    realized_vol, dec_price_change = get_data()
    logger.info(f"Input data: {realized_vol}, {dec_price_change}")
    X = process_data(realized_vol, dec_price_change)
    contracts = {
        "nft_manager": nft_manager_address,
        "tokenA": tokenA_address,
        "tokenB": tokenB_address,
        "pool": pool_address,
    }
    agent = create_agent(
        model_id=pred_model_id,
        version_id=pred_version_id,
        chain=chain,
        contracts=contracts,
        account=account,
    )
    result = predict(agent, X)
    predicted_value = get_pred_val(result)
    logger.info(f"Result: {result}")
    with agent.execute() as contracts:
        logger.info("Executing contract")
        if nft_id is None:
            positions = [
                max(get_all_user_positions(contracts.nft_manager, user_address))
            ]
        else:
            positions = [nft_id]
        logger.info(f"Closing the following positions {positions}")
        for nft_id in positions:
            close_position(user_address, contracts.nft_manager, nft_id)
        logger.info("Calculating mint params...")
        _, curr_tick, _, _, _, _, _ = contracts.pool.slot0()
        if not check_allowance(
            contracts.tokenA, nft_manager_address, account, tokenA_amount
        ):
            approve_token(contracts.tokenA, nft_manager_address, tokenA_amount)
        if not check_allowance(
            contracts.tokenB, nft_manager_address, account, tokenB_amount
        ):
            approve_token(contracts.tokenB, nft_manager_address, tokenB_amount)
        tokenA_decimals = contracts.tokenA.decimals()
        tokenB_decimals = contracts.tokenB.decimals()
        predicted_value = predicted_value / 100 * 1.96  # convert to decimal %
        lower_tick, upper_tick = get_tick_range(
            curr_tick, predicted_value, tokenA_decimals, tokenB_decimals, pool_fee
        )
        mint_params = get_mint_params(
            user_address,
            contracts.tokenA.address,
            contracts.tokenB.address,
            tokenA_amount,
            tokenB_amount,
            pool_fee,
            lower_tick,
            upper_tick,
        )
        # step 5: mint new position
        logger.info("Minting new position...")
        contract_result = contracts.nft_manager.mint(mint_params)
        logger.info("SUCCESSFULLY MINTED A POSITION")
        logger.info("Contract executed")

    logger.info(f"Contract result: {contract_result}")
    pprint.pprint(contract_result.__dict__)
    logger.info("Finished")


In [ ]:

MODEL_ID = 858
VERSION_ID = 1
tokenA_amount = 500
tokenB_amount = 500

rebalance_lp(tokenA_amount, tokenB_amount, MODEL_ID, VERSION_ID, account="giza-agent-test-1")

## Benchmarks

In [ ]:
## neural network 
# - hidden layer with shape [32, 16]
#     - transpile from onnx: ~1sec
#     - inference: ~1sec
#     - generate proof: 37.38549 secs
#     - verify proof: 0.974234442 secs

# - hidden layer with shape [64, 32]
#     - transpile from onnx: 42.91086792945862 secs
#     - inference: 4.4827117919921875 secs
#     - generate proof: 78.88475 secs
#     - verify proof: 2.013659523 secs

# - hidden layer with shape [128, 64]
#     - transpile from onnx: 53.90036916732788 secs
#     - inference: 7.7867348194122314 secs
#     - generate proof: out of memory